In [ ]:
import yfinance as yf
import pandas as pd
# from config import nikkei_225_tickers
# 日经200的股票列表
nikkei_225_tickers = [
    "1332.T", "1333.T", "1605.T", "1721.T", "1801.T", "1802.T", "1803.T", "1808.T", "1812.T", "1925.T",
    "1928.T", "1963.T", "2002.T", "2269.T", "2282.T", "2413.T", "2432.T", "2768.T", "2801.T", "2802.T",
    "2871.T", "2914.T", "3086.T", "3099.T", "3101.T", "3103.T", "3105.T", "3110.T", "3289.T", "3382.T",
    "3401.T", "3402.T", "3405.T", "3407.T", "3436.T", "3861.T", "3863.T", "4004.T", "4005.T", "4021.T",
    "4042.T", "4043.T", "4061.T", "4063.T", "4151.T", "4183.T", "4188.T", "4208.T", "4272.T", "4324.T",
    "4452.T", "4503.T", "4506.T", "4507.T", "4519.T", "4523.T", "4543.T", "4568.T", "4578.T", "4631.T",
    "4661.T", "4689.T", "4704.T", "4751.T", "4755.T", "4901.T", "4902.T", "4911.T", "4912.T", "5019.T",
    "5020.T", "5101.T", "5108.T", "5201.T", "5202.T", "5214.T", "5232.T", "5233.T", "5301.T", "5332.T",
    "5333.T", "5401.T", "5406.T", "5411.T", "5541.T", "5631.T", "5703.T", "5706.T", "5711.T", "5713.T",
    "5714.T", "5801.T", "5802.T", "5803.T", "5901.T", "5902.T", "5943.T", "6103.T", "6113.T", "6301.T",
    "6302.T", "6305.T", "6326.T", "6361.T", "6366.T", "6367.T", "6471.T", "6472.T", "6473.T", "6479.T",
    "6501.T", "6503.T", "6504.T", "6506.T", "6645.T", "6674.T", "6701.T", "6702.T", "6703.T", "6723.T",
    "6724.T", "6752.T", "6758.T", "6762.T", "6770.T", "6841.T", "6856.T", "6861.T", "6902.T", "6952.T",
    "6954.T", "6971.T", "6976.T", "6988.T", "7003.T", "7004.T", "7011.T", "7012.T", "7013.T", "7014.T",
    "7186.T", "7201.T", "7202.T", "7203.T", "7205.T", "7211.T", "7261.T", "7267.T", "7269.T", "7270.T",
    "7272.T", "7731.T", "7733.T", "7735.T", "7741.T", "7747.T", "7751.T", "7752.T", "7762.T", "7832.T",
    "7911.T", "7912.T", "7951.T", "7956.T", "8001.T", "8002.T", "8015.T", "8028.T", "8031.T", "8035.T",
    "8053.T", "8058.T", "8233.T", "8242.T", "8252.T", "8253.T", "8267.T", "8270.T", "8303.T", "8304.T",
    "8306.T", "8308.T", "8309.T", "8316.T", "8331.T", "8354.T", "8355.T", "8411.T", "8601.T", "8604.T",
    "8628.T", "8630.T", "8725.T", "8729.T", "8750.T", "8766.T", "8795.T", "8801.T", "8802.T", "8804.T",
    "8830.T", "8831.T", "8834.T", "9001.T", "9005.T", "9007.T", "9008.T", "9009.T", "9020.T", "9021.T",
    "9022.T", "9062.T", "9064.T", "9101.T", "9104.T", "9107.T", "9109.T", "9201.T", "9202.T", "9301.T",
    "9412.T", "9432.T", "9433.T", "9434.T", "9501.T", "9502.T", "9503.T", "9531.T", "9532.T", "9602.T",
    "9613.T", "9681.T", "9735.T", "9766.T", "9783.T", "9810.T", "9831.T", "9983.T", "9984.T"
]

# 定义巴菲特的选股标准（适当放松一些条件）
def is_potential_stock(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    balance_sheet = stock.balance_sheet
    cash_flow = stock.cashflow

    # 确保数据不为空
    if financials.empty or balance_sheet.empty or cash_flow.empty:
        print(f"{ticker}: 財務データが不完全です。")
        return False

    # 检查盈利能力，只要最近两年中至少一年盈利增长即可
    earnings_growth = financials.loc['Net Income'].pct_change(fill_method=None).dropna()
    if earnings_growth.empty or (len(earnings_growth) > 1 and earnings_growth.iloc[-1] < 0 and earnings_growth.iloc[-2] < 0):
        print(f"{ticker}: 収益成長率がマイナスです。")
        return False

    # 检查股东权益回报率（ROE），降低标准至10%
    roe = financials.loc['Net Income'] / balance_sheet.loc['Total Stockholder Equity']
    if roe.mean() < 0.10:
        print(f"{ticker}: ROEが10％未満です。")
        return False

    # 检查负债水平，负债比率标准放宽至2
    debt_to_equity = balance_sheet.loc['Total Liabilities'] / balance_sheet.loc['Total Stockholder Equity']
    if debt_to_equity.mean() > 2:
        print(f"{ticker}: 負債比率が2を超えています。")
        return False

    # 检查现金流，只要过去两年中至少一年为正即可
    operating_cash_flow = cash_flow.loc['Total Cash From Operating Activities']
    if operating_cash_flow.empty or (len(operating_cash_flow) > 1 and operating_cash_flow.iloc[-1] < 0 and operating_cash_flow.iloc[-2] < 0):
        print(f"{ticker}: 営業キャッシュフローがマイナスです。")
        return False

    return True

# 扫描日经225的股票，选出有潜力的股票
potential_stocks = []

for ticker in nikkei_225_tickers:
    try:
        print(f"{ticker}の処理を開始します...")
        if is_potential_stock(ticker):
            print(f"{ticker}: すべての基準をクリアしました。")
            potential_stocks.append(ticker)
        else:
            print(f"{ticker}: 基準をクリアしませんでした。")
    except Exception as e:
        print(f"{ticker}の処理中にエラーが発生しました: {e}")

# 输出有潜力的股票列表

In [13]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import os

# 手动添加日经225指数成分股的股票代码列表
nikkei_225_tickers = [
    "1332.T", "1333.T", "1605.T", "1721.T", "1801.T", "1802.T", "1803.T", "1808.T", "1812.T", "1925.T",
    "1928.T", "1963.T", "2002.T", "2269.T", "2282.T", "2413.T", "2432.T", "2768.T", "2801.T", "2802.T",
    "2871.T", "2914.T", "3086.T", "3099.T", "3101.T", "3103.T", "3105.T", "3110.T", "3289.T", "3382.T",
    "3401.T", "3402.T", "3405.T", "3407.T", "3436.T", "3861.T", "3863.T", "4004.T", "4005.T", "4021.T",
    "4042.T", "4043.T", "4061.T", "4063.T", "4151.T", "4183.T", "4188.T", "4208.T", "4272.T", "4324.T",
    "4452.T", "4503.T", "4506.T", "4507.T", "4519.T", "4523.T", "4543.T", "4568.T", "4578.T", "4631.T",
    "4661.T", "4689.T", "4704.T", "4751.T", "4755.T", "4901.T", "4902.T", "4911.T", "4912.T", "5019.T",
    "5020.T", "5101.T", "5108.T", "5201.T", "5202.T", "5214.T", "5232.T", "5233.T", "5301.T", "5332.T",
    "5333.T", "5401.T", "5406.T", "5411.T", "5541.T", "5631.T", "5703.T", "5706.T", "5711.T", "5713.T",
    "5714.T", "5801.T", "5802.T", "5803.T", "5901.T", "5902.T", "5943.T", "6103.T", "6113.T", "6301.T",
    "6302.T", "6305.T", "6326.T", "6361.T", "6366.T", "6367.T", "6471.T", "6472.T", "6473.T", "6479.T",
    "6501.T", "6503.T", "6504.T", "6506.T", "6645.T", "6674.T", "6701.T", "6702.T", "6703.T", "6723.T",
    "6724.T", "6752.T", "6758.T", "6762.T", "6770.T", "6841.T", "6856.T", "6861.T", "6902.T", "6952.T",
    "6954.T", "6971.T", "6976.T", "6988.T", "7003.T", "7004.T", "7011.T", "7012.T", "7013.T", "7014.T",
    "7186.T", "7201.T", "7202.T", "7203.T", "7205.T", "7211.T", "7261.T", "7267.T", "7269.T", "7270.T",
    "7272.T", "7731.T", "7733.T", "7735.T", "7741.T", "7747.T", "7751.T", "7752.T", "7762.T", "7832.T",
    "7911.T", "7912.T", "7951.T", "7956.T", "8001.T", "8002.T", "8015.T", "8028.T", "8031.T", "8035.T",
    "8053.T", "8058.T", "8233.T", "8242.T", "8252.T", "8253.T", "8267.T", "8270.T", "8303.T", "8304.T",
    "8306.T", "8308.T", "8309.T", "8316.T", "8331.T", "8354.T", "8355.T", "8411.T", "8601.T", "8604.T",
    "8628.T", "8630.T", "8725.T", "8729.T", "8750.T", "8766.T", "8795.T", "8801.T", "8802.T", "8804.T",
    "8830.T", "8831.T", "8834.T", "9001.T", "9005.T", "9007.T", "9008.T", "9009.T", "9020.T", "9021.T",
    "9022.T", "9062.T", "9064.T", "9101.T", "9104.T", "9107.T", "9109.T", "9201.T", "9202.T", "9301.T",
    "9412.T", "9432.T", "9433.T", "9434.T", "9501.T", "9502.T", "9503.T", "9531.T", "9532.T", "9602.T",
    "9613.T", "9681.T", "9735.T", "9766.T", "9783.T", "9810.T", "9831.T", "9983.T", "9984.T"
]

# 定义计算技术指标的函数
def calculate_indicators(df):
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA10'] = df['Close'].rolling(window=10).mean()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    return df

# 筛选股票的策略
def is_potential_stock(df):
    latest = df.iloc[-1]
    if latest['MA5'] > latest['MA10'] and latest['RSI'] < 70 and latest['MACD'] > latest['MACD_signal']:
        return True
    return False

# 绘制技术指标图像
def plot_indicators(df, ticker):
    plt.figure(figsize=(14, 7))

    # Close price and moving averages
    plt.subplot(3, 1, 1)
    plt.plot(df['Close'], label='Close')
    plt.plot(df['MA5'], label='MA5')
    plt.plot(df['MA10'], label='MA10')
    plt.title(f'{ticker} - Close Price and Moving Averages')
    plt.legend()

    # RSI
    plt.subplot(3, 1, 2)
    plt.plot(df['RSI'], label='RSI')
    plt.axhline(70, color='r', linestyle='--')
    plt.axhline(30, color='g', linestyle='--')
    plt.title(f'{ticker} - RSI')
    plt.legend()

    # MACD
    plt.subplot(3, 1, 3)
    plt.plot(df['MACD'], label='MACD')
    plt.plot(df['MACD_signal'], label='MACD Signal')
    plt.title(f'{ticker} - MACD')
    plt.legend()

    plt.tight_layout()
    plt.savefig(f'{ticker}_indicators.png')
    plt.close()

# 获取每只股票的数据并计算技术指标
potential_stocks = []

for ticker in nikkei_225_tickers:
    try:
        df = yf.download(ticker, period="1y")
        df = calculate_indicators(df)
        if is_potential_stock(df):
            potential_stocks.append(ticker)
            plot_indicators(df, ticker)
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# 打印选出的股票的公司名并显示财务指标
for ticker in potential_stocks:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        financials = stock.financials
        print(f"\n{ticker} - {info['longName']}")

        # 最近一周的技术指标图像
        os.system(f"open {ticker}_indicators.png")

        # 近两年的财务指标
        print("最近两年的财务指标:")
        print(financials.loc[:, financials.columns[:8]])
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Error processing 8028.T: single positional indexer is out-of-bounds


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['8270.T']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (period=1y)')


$8270.T: possibly delisted; No price data found  (period=1y)
Error processing 8270.T: single positional indexer is out-of-bounds


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['8303.T']: YFChartError('%ticker%: No data found, symbol may be delisted')


...

Error processing 9783.T: single positional indexer is out-of-bounds


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['9810.T']: YFChartError('%ticker%: No data found, symbol may be delisted')


Error processing 9810.T: single positional indexer is out-of-bounds


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



3401.T - Teijin Limited
最近两年的财务指标:
                                                         2024-03-31  \
Tax Effect Of Unusual Items                            4888008000.0   
Tax Rate For Calcs                                            0.232   
Normalized EBITDA                                     97009000000.0   
Total Unusual Items                                   21069000000.0   
Total Unusual Items Excluding Goodwill                21069000000.0   
Net Income From Continuing Operation Net Minori...    10599000000.0   
Reconciled Depreciation                               78885000000.0   
Reconciled Cost Of Revenue                           757000000000.0   
EBITDA                                               118078000000.0   
EBIT                                                  39193000000.0   
Net Interest Income                                   -8152000000.0   
Interest Expense                                      10782000000.0   
Interest Income                          